## Monopoly Game Agent (v2)

**Proof of Concept**: Connecting LLM I/O to game simulators

The following notebook contains:
- Function wrappers to retrieve game state
- Basic prompt template that we can inject state into
- Output parser to extract necessary data

Note: You can download and import into Goolge Colab if that is preferred.

---

### Project Setup
Install dependencies

In [2]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_community-0.3.13-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain-0.3.13-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached langchain_core-0.3.28-py3-none-any.whl.metadata (6.3 kB)
  Using cached langsmith-0.2.4-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic_settings-2.7.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached tiktoken

Import monopoly simulator

(Note for future: We should install the game simulator as a package)

In [1]:
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

Importing other dependencies

In [4]:
# LangChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI, LLMChain, PromptTemplate

# other imports
from pydantic import BaseModel, Field
from typing import List

# dotenv for loading API keys
import os
from dotenv import load_dotenv

In [6]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

### Game Functions
Wrappers that retrieves relevant game state(s)

In [7]:
def initialize_game() -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    player1 = Player('player1', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('player2', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2] # For now, player 1 always comes first
    }

In [8]:
def get_current_state(players) -> dict:
    """
    Retrieves the current state of each player, including position, owned roads, 
    money, mortgaged properties, and other status details.

    Args:
        players (list[Player]): List of Player objects in the game.

    Returns:
        dict: A dictionary containing:
            - "players": A list of dictionaries, each with a player's state.
    """
    
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

In [15]:
# Example usage of the above function
game = initialize_game()
game["bank"]

{'cash': 5000, 'houses': 32, 'hotels': 12}

### Prompt Template

The following defines a customizable prompt template for an agent in a Monopoly game. Each part of the template is easily customizable using placeholders for various game elements.

In [12]:
# The agent plays as Player 1 by default
agent_role = "Player 1" 

In [13]:
# prompt template wrapper / hook, a function that returns a string
def prompt_template():
    """
    Generates a formatted prompt string for an agent in a Monopoly game, detailing 
    the game's current state and guiding strategic decision-making.

    Returns:
        str: A prompt template string with placeholders for:
            - {agent_role}: The role of the agent in the game.
            - {initial_bank}: Initial bank details.
            - {initial_board}: Initial board configuration.
            - {initial_roads}: List of roads.
            - {initial_properties}: List of properties.

    Usage:
        Substitute placeholders to customize the prompt with the game state.
    """
    
    return  """
        You are the {agent_role} in a Monopoly game. Here is the current game state:

        Bank:
        {initial_bank}

        Board:
        {initial_board}

        Roads:
        {initial_roads}

        Properties:
        {initial_properties}

        Players:
        Player 1 and Player 2

        Your Objective:
        Given the current state of the game, make strategic moves that maximizes your chances of winning.

        Guidelines:
        1. Analyze each component of the game state to understand your current situation.
        2. Consider any immediate risks or opportunities from property ownership, player positions, or your current balance.

        Instructions:
        - Reason step-by-step to ensure your action aligns with the game’s rules and overall strategy.
        - Provide your next move by determining if you should buy the property or not (yes or no)
  """

In [16]:
# Here's some example usage of the above function. We'll overwrite 
# these variables later in actual use

# Define the game setup and get initial game state
game = initialize_game()  # Initializes the bank, board, roads, properties, and players

# Generate the prompt with specific game details
template = prompt_template()
formatted_prompt = template.format(
    agent_role="Player 1",
    initial_bank=game["bank"],
    initial_board=game["board"],
    initial_roads=game["roads"],
    initial_properties=game["properties"]
)

print(formatted_prompt)


        You are the Player 1 in a Monopoly game. Here is the current game state:

        Bank:
        {'cash': 5000, 'houses': 32, 'hotels': 12}

        Board:
        [{'name': 'go', 'type': 'go'}, {'name': 'old kent road', 'type': 'road'}, {'name': 'community chest', 'type': 'community chest'}, {'name': 'whitechapel road', 'type': 'road'}, {'name': 'income tax', 'type': 'tax'}, {'name': 'kings cross station', 'type': 'station'}, {'name': 'the angel islington', 'type': 'road'}, {'name': 'chance', 'type': 'chance'}, {'name': 'euston road', 'type': 'road'}, {'name': 'pentonville road', 'type': 'road'}, {'name': 'jail', 'type': 'jail'}, {'name': 'pall mall', 'type': 'road'}, {'name': 'Electric company', 'type': 'utility'}, {'name': 'whitehall', 'type': 'road'}, {'name': 'northumberland avenue', 'type': 'road'}, {'name': 'marylebone station', 'type': 'station'}, {'name': 'bow street', 'type': 'road'}, {'name': 'community chest', 'type': 'community chest'}, {'name': 'marlborough street

### Output Parser

The following defines a parser for interpreting the agent's output in the Monopoly game

In [17]:
class Output(BaseModel):
    reasoning: str = Field(description="Your reasoning for the decision")
    decision: str = Field(description="Your decision for the next move")

In [18]:
def output_parser(model):
    return model.with_structured_output(Output)

### Simulate the Game

Set up prompt template and LLM chain

In [19]:
model = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

In [20]:
structured_llm = model.with_structured_output(Output)

Initialize the game and make arbitrary moves

In [21]:
game = initialize_game()

In [22]:
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

stop_at_round = 5 # arbitrary number of rounds to play before agent comes in and make a decision (for POC)

In [23]:
idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < stop_at_round:
    for player in list_players:
        player.play()
    idx_count += 1

injecting variables

1. Set up prompt template and LLM chain
2. Hardcode some injection variables & make sure it works
3. Code to retrieve game info / states

In [24]:
initial_template = prompt_template()

In [25]:
### Only one turn of the game is played so far
context = initial_template.format(
    agent_role="Player 1",  # or as appropriate
    initial_bank=game["bank"],
    initial_board=game["board"],
    initial_roads=game["roads"],
    initial_properties=game["properties"]
)

In [26]:
response = structured_llm.invoke(f"${context}. player_state is ${get_current_state(list_players)}")

In [27]:
response.decision

'no'

In [28]:
response.reasoning

"As Player 1, I currently have 302 in cash and I need to consider my options carefully. I own several properties but have no houses or hotels built on them yet, which limits my income potential. Player 2 owns 'whitechapel road', which I land on if I move 6 spaces from my current position (39). However, I also need to think about acquiring more properties to increase my chances of generating income. The properties available on the board include several that are not owned, such as 'the angel islington' and 'euston road', which I already own. However, buying properties requires cash, and I need to ensure I have enough to cover any rents. Given that I can only land on properties not owned by others and I need to manage my cash flow effectively, I should not buy any additional properties now. My current cash position is low, and I need to protect it for potential rent payments or other expenses."